# Reproject script 
Simple script to reproject all the maps to the N2D+ pixel grid (no smoothing)

In [1]:
from reproject import reproject_interp
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS

In [5]:
file_n2dp =     f'{DATA_PATH}/Galactic/NOEMA/data/CloudH_lines_moms/CloudH_N2Dp_L18-merged_mom0.fits'
file_n2dp_err = f'{DATA_PATH}/Galactic/NOEMA/data/CloudH_lines_moms/CloudH_N2Dp_L18-merged_mom0err.fits'
file_n2hp =     f'{DATA_PATH}/Galactic/NOEMA/data/CloudH_lines_moms/CloudH_N2Hp_L30-merged_iso_mom0.fits'
file_n2hp_err = f'{DATA_PATH}/Galactic/NOEMA/data/CloudH_lines_moms/CloudH_N2Hp_L30-merged_iso_mom0err.fits'
file_dcop =     f'{DATA_PATH}/Galactic/NOEMA/data/CloudH_lines_moms/CloudH_DCOp_L09-merged_mom0.fits'
file_c18o =     f'{DATA_PATH}/Galactic/GBT/Cloud_H/ClH_C18O_bl_40to50kms_mom0.fits'
file_c18o_err = f'{DATA_PATH}/Galactic/GBT/Cloud_H/ClH_C18O_bl_40to50kms_mom0err.fits'
file_tkin =     f'{DATA_PATH}/Galactic/VLA/cloudH_temp/figure7_tkin_F2.fits'
file_msd =      f'{DATA_PATH}/Galactic/KT_extinction_maps/new/data/cloudH.msd.fits'
file_fd =       f'./data/fpD_H11.fits'

hdu_n2dp = fits.open(file_n2dp)[0]
hdu_n2dp_err = fits.open(file_n2dp_err)[0]
hdu_n2hp = fits.open(file_n2hp)[0]
hdu_n2hp_err = fits.open(file_n2hp_err)[0]
hdu_dcop = fits.open(file_dcop)[0]
hdu_c18o = fits.open(file_c18o)[0]
hdu_tkin = fits.open(file_tkin)[0]
hdu_msd = fits.open(file_msd)[0]
hdu_fd = fits.open(file_fd)[0]

# Remove addition header information
hdu_tkin.data = hdu_tkin.data[0]
del hdu_tkin.header['*3*']
hdu_tkin.header['WCSAXES'] = 2

# Fix header
header_tmp = WCS(hdu_fd.header).to_header()
for key in header_tmp:
    hdu_fd.header[key] = header_tmp[key]

In [6]:
# reproject the N2H+ and C18O onto the N2D+ grid
array_n2hp, footprint = reproject_interp(hdu_n2hp, hdu_n2dp.header)
array_n2hp_err, footprint = reproject_interp(hdu_n2hp_err, hdu_n2dp.header)
array_dcop, footprint = reproject_interp(hdu_dcop, hdu_n2dp.header)
array_c18o, footprint = reproject_interp(hdu_c18o, hdu_n2dp.header)
array_tkin, footprint = reproject_interp(hdu_tkin, hdu_n2dp.header)
array_msd, footprint = reproject_interp(hdu_msd, hdu_n2dp.header)
array_fd, footprint = reproject_interp(hdu_fd, hdu_n2dp.header, order=0)

Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [7]:
# make new headers using the old header and only replacing the CDELT, CRPIX, and NAXIS info
header_n2dp = hdu_n2dp.header
header_dcop = hdu_dcop.header
header_n2hp = hdu_n2hp.header
header_n2hp_err = hdu_n2hp_err.header
header_c18o = hdu_c18o.header
header_tkin = hdu_tkin.header
header_msd = hdu_msd.header
header_fd = hdu_fd.header

for key in header_n2dp:
    if key in ['CDELT1', 'CDELT2', 'CRPIX1', 'CRPIX2', 'NAXIS1', 'NAXIS2', 'CTYPE1', 'CTYPE2', 'CUNIT1', 'CUNIT2', 'CRVAL1', 'CRVAL2']:
        header_n2hp[key] = header_n2dp[key]
        header_n2hp_err[key] = header_n2dp[key]
        header_dcop[key] = header_n2dp[key]
        header_c18o[key] = header_n2dp[key]
        header_tkin[key] = header_n2dp[key]
        header_msd[key] = header_n2dp[key]
        header_fd[key] = header_n2dp[key]

# make hdus 
hdu_n2hp = fits.PrimaryHDU(array_n2hp, header_n2hp)
hdu_n2hp_err = fits.PrimaryHDU(array_n2hp_err, header_n2hp_err)
hdu_dcop = fits.PrimaryHDU(array_dcop, header_dcop)
hdu_c18o = fits.PrimaryHDU(array_c18o, header_c18o)
hdu_tkin = fits.PrimaryHDU(array_tkin, header_tkin)
hdu_msd = fits.PrimaryHDU(array_msd, header_msd)
hdu_fd = fits.PrimaryHDU(array_fd, header_fd)

# write out the new files
hdu_n2dp.writeto('./data/n2dp_mom0.fits', overwrite=True)
hdu_n2dp_err.writeto('./data/n2dp_mom0err.fits', overwrite=True)
hdu_dcop.writeto('./data/dcop_mom0_reprojected.fits', overwrite=True)
hdu_n2hp.writeto('./data/n2hp_mom0_reprojected.fits', overwrite=True)
hdu_n2hp_err.writeto('./data/n2hp_mom0err_reprojected.fits', overwrite=True)
hdu_c18o.writeto('./data/c18o_mom0_reprojected.fits', overwrite=True)
hdu_tkin.writeto('./data/tkin_reprojected.fits', overwrite=True)
hdu_msd.writeto('./data/msd_reprojected.fits', overwrite=True)
hdu_fd.writeto('./data/fpD_H11_reprojected.fits', overwrite=True)